In [2]:
import os
import subprocess
import json
from datetime import datetime
from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel
from typing import List, Dict, Any
import sqlite3
import requests
from openai import OpenAI


In [3]:
import sys
import warnings
import asyncio
import logging

from datetime import datetime

# from latest_ai_development.crew import LatestAiDevelopment

warnings.filterwarnings("ignore", category=SyntaxWarning, module="pysbd")

from fastapi import FastAPI, Request, HTTPException
from pydantic import BaseModel
import uvicorn
import openai
import logging


In [4]:
api_key = os.environ("API_KEY")

TypeError: '_Environ' object is not callable

In [5]:
url = 'https://aiproxy.sanand.workers.dev/openai/v1/chat/completions'

In [ ]:
headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

In [ ]:
content = ''

In [ ]:
payload = {
        "model": "gpt-4o-mini",
        "response_format": { "type": "json_object" },
        "messages": [
            {
                "role": "system",
                "content": """
Your role is to run the datagen.py script
"""
            },
            {
                "role": "user",
                "content": content
            }
        ]
    }

In [ ]:
response = requests.post(
    url = url,
    headers= headers,
    json = payload
)

In [1]:
# FastAPI app definition
app = FastAPI()

NameError: name 'FastAPI' is not defined

In [ ]:
@app.post("/run")
async def run_task(task: str):
    logging.info(f"Running task: {str}")
    try:
        # Run the crew kickoff asynchronously
        asyncio.create_task(run_crew(task))
        return {"message": f"Executing task: {task}"}
    except Exception as e:
        return {"error": str(e)}
    

async def run_task(task: str = Query(...)):
    try:
        steps = parse_task_with_llm(task)
        execute_steps(steps)
        return {"status": "success"}
    except HTTPException as he:
        raise he
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [ ]:
async def run_crew(task: str):
    """
    This will run the crew in a separate thread to avoid blocking FastAPI.
    """
    logging.info(f"Starting crew task: {task}")
    try:
        # Pass the task input to the crew kickoff
        inputs = {'topic': task}
        LatestAiDevelopment().crew().kickoff(inputs=inputs)
    except Exception as e:
        logging.error(f"Error during crew kickoff: {str(e)}")


In [ ]:
# Endpoint to read a file
@app.get("/read")
async def read_file(path: str):
    try:
        with open(path, 'r') as file:
            content = file.read()
        return {"content": content}
    except Exception as e:
        logging.error(f"Error reading file {path}: {str(e)}")
        raise HTTPException(status_code=400, detail=f"Error reading file: {str(e)}")